### Importing

In [1]:
from utils import dataGetter
from datetime import datetime
import pandas as pd
import numpy as np


### Getting data, data will be stored in /tmp/history.zip on a Unix base file system

In [26]:
data = dataGetter.getStockData(datetime(2017,11,14))
#data = dataGetter.getDataFromFile("/path/to/history.zip")

### Getting floating data

In [52]:
float_data = pd.read_csv( 'data/vn30_floating.csv',index_col=0,dtype={'totalShares':np.int64,'floating':np.int64})

vn30_members = float_data.index.values
vn30_data = data[(data.Ticker.isin(vn30_members))]

float_data['non_float'] = float_data['totalShares'] - float_data['floating']
float_data['f_ratio_0'] = float_data['floating']/ float_data['totalShares']
float_data['f_ratio_1'] = (float_data['floating'] - float_data['non_float']) / float_data['floating']


In [53]:
#create a copy, not a slice of old data
vn30_data = vn30_data.copy()
vn30_data['FloatingShares'] = vn30_data['Ticker'].map(float_data['floating'])
vn30_data['TotalShares'] = vn30_data['Ticker'].map(float_data['totalShares'])
vn30_data['MktCap'] = vn30_data['Close']*vn30_data['TotalShares']
vn30_data['f_0'] = vn30_data['Ticker'].map(float_data['f_ratio_0'])
vn30_data['f_1'] = vn30_data['Ticker'].map(float_data['f_ratio_1'])


### Calculating $c_i$

In [54]:
#use the business day before the third monday of July
calcDate = datetime(2017,7,14)
calcData = vn30_data[vn30_data['Date'] == calcDate]

stop = False
Z = 0.1
c = pd.Series(1, index=calcData.index)

while not stop:
    totalMktCap = (calcData['MktCap']*c).sum()
    mktCap = calcData['MktCap']*c
    mktCap_pct = mktCap / totalMktCap
    tickers = calcData['Ticker'].values
    weightMap = dict(zip(tickers, mktCap_pct.values))
    mktCapMap = dict(zip(tickers, mktCap.values))

    J = [x for x in weightMap if weightMap[x] <= Z ]
    I = calcData[calcData['Ticker'].isin(J) ]['MktCap'].sum()/totalMktCap    
    nJ = [x for x in weightMap if weightMap[x]>Z]
    if( len( nJ ) == 0 ):
        stop = True
    psf = calcData['Close']*calcData['f_0']*calcData['TotalShares']
    psf_J = psf[calcData['Ticker'].isin(J)].sum()
    new_c = Z/I*psf_J/psf[calcData['Ticker'].isin(nJ)]
    #print new_c
    c[new_c.index] = new_c
#c

In [55]:
tickers = calcData['Ticker']
c_map = dict( zip( tickers.values, c.values ))

In [56]:
cmv = vn30_data['Close'] * vn30_data['TotalShares'] * vn30_data['f_0']* vn30_data['Ticker'].map(c_map)
vn30_data['CMV'] = cmv

In [59]:
#vn30_data['vn30_comp'] = vn30_data['Close'] * vn30_data['IndexWeight']
vn30_data.groupby(['Date'])['CMV'].sum()/1.713809e+09

Date
2000-07-28      0.327779
2000-07-31      0.333912
2000-08-02      0.340063
2000-08-04      0.346214
2000-08-07      0.352347
2000-08-09      0.358498
2000-08-11      0.364649
2000-08-14      0.370800
2000-08-16      0.376933
2000-08-18      0.383084
2000-08-21      0.389235
2000-08-23      0.395368
2000-08-25      0.389235
2000-08-28      0.383084
2000-08-30      0.376933
2000-09-01      0.370800
2000-09-06      0.364649
2000-09-08      0.358498
2000-09-11      0.364649
2000-09-13      0.370800
2000-09-15      0.376933
2000-09-18      0.370800
2000-09-20      0.376933
2000-09-22      0.370800
2000-09-25      0.370800
2000-09-27      0.370800
2000-09-29      0.372844
2000-10-02      0.372844
2000-10-04      0.372844
2000-10-06      0.372844
                 ...    
2017-10-04    770.231223
2017-10-05    770.301772
2017-10-06    772.416213
2017-10-09    773.156049
2017-10-10    774.842695
2017-10-11    779.125500
2017-10-12    782.119766
2017-10-13    789.588085
2017-10-16    786.41